![image info](https://raw.githubusercontent.com/albahnsen/MIAD_ML_and_NLP/main/images/banner_1.png)

# Proyecto 2 - Clasificación de género de películas

El propósito de este proyecto es que puedan poner en práctica, en sus respectivos grupos de trabajo, sus conocimientos sobre técnicas de preprocesamiento, modelos predictivos de NLP, y la disponibilización de modelos. Para su desarrollo tengan en cuenta las instrucciones dadas en la "Guía del proyecto 2: Clasificación de género de películas"

**Entrega**: La entrega del proyecto deberán realizarla durante la semana 8. Sin embargo, es importante que avancen en la semana 7 en el modelado del problema y en parte del informe, tal y como se les indicó en la guía.

Para hacer la entrega, deberán adjuntar el informe autocontenido en PDF a la actividad de entrega del proyecto que encontrarán en la semana 8, y subir el archivo de predicciones a la [competencia de Kaggle](https://www.kaggle.com/t/2c54d005f76747fe83f77fbf8b3ec232).

## Datos para la predicción de género en películas

![image info](https://raw.githubusercontent.com/albahnsen/MIAD_ML_and_NLP/main/images/moviegenre.png)

En este proyecto se usará un conjunto de datos de géneros de películas. Cada observación contiene el título de una película, su año de lanzamiento, la sinopsis o plot de la película (resumen de la trama) y los géneros a los que pertenece (una película puede pertenercer a más de un género). Por ejemplo:
- Título: 'How to Be a Serial Killer'
- Plot: 'A serial killer decides to teach the secrets of his satisfying career to a video store clerk.'
- Generos: 'Comedy', 'Crime', 'Horror'

La idea es que usen estos datos para predecir la probabilidad de que una película pertenezca, dada la sinopsis, a cada uno de los géneros.

Agradecemos al profesor Fabio González, Ph.D. y a su alumno John Arevalo por proporcionar este conjunto de datos. Ver https://arxiv.org/abs/1702.01992

## Ejemplo predicción conjunto de test para envío a Kaggle
En esta sección encontrarán el formato en el que deben guardar los resultados de la predicción para que puedan subirlos a la competencia en Kaggle.

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Importación librerías
import pandas as pd
import numpy as np
import ast
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

In [3]:
# Carga de datos de archivo .csv
dataTraining = pd.read_csv('https://github.com/albahnsen/MIAD_ML_and_NLP/raw/main/datasets/dataTraining.zip', encoding='UTF-8', index_col=0)
dataTesting = pd.read_csv('https://github.com/albahnsen/MIAD_ML_and_NLP/raw/main/datasets/dataTesting.zip', encoding='UTF-8', index_col=0)

In [4]:
# Visualización datos de entrenamiento
dataTraining.head()

,year,title,plot,genres,rating
3107,2003,Most,most is the story of a single father who takes...,"['Short', 'Drama']",8.0
900,2008,How to Be a Serial Killer,a serial killer decides to teach the secrets o...,"['Comedy', 'Crime', 'Horror']",5.6
6724,1941,A Woman's Face,"in sweden , a female blackmailer with a disfi...","['Drama', 'Film-Noir', 'Thriller']",7.2
4704,1954,Executive Suite,"in a friday afternoon in new york , the presi...",['Drama'],7.4
2582,1990,Narrow Margin,"in los angeles , the editor of a publishing h...","['Action', 'Crime', 'Thriller']",6.6


In [5]:
# Visualización datos de test
dataTesting.head()

,year,title,plot
1,1999,Message in a Bottle,"who meets by fate , shall be sealed by fate ...."
4,1978,Midnight Express,"the true story of billy hayes , an american c..."
5,1996,Primal Fear,martin vail left the chicago da ' s office to ...
6,1950,Crisis,husband and wife americans dr . eugene and mr...
7,1959,The Tingler,the coroner and scientist dr . warren chapin ...


In [6]:
# Finding unique genres
movie_genre = list(dataTraining['genres'].unique())
movie_genre.sort()
movie_genre

["['Action', 'Adventure', 'Biography', 'Crime', 'History', 'Romance', 'Western']",
 "['Action', 'Adventure', 'Biography', 'Drama', 'History', 'Romance', 'War']",
 "['Action', 'Adventure', 'Biography', 'Drama', 'History', 'Thriller']",
 "['Action', 'Adventure', 'Biography', 'Drama', 'History', 'War']",
 "['Action', 'Adventure', 'Comedy', 'Crime', 'Drama', 'Romance']",
 "['Action', 'Adventure', 'Comedy', 'Crime', 'Drama', 'Thriller']",
 "['Action', 'Adventure', 'Comedy', 'Crime', 'Drama']",
 "['Action', 'Adventure', 'Comedy', 'Crime', 'Music', 'Mystery']",
 "['Action', 'Adventure', 'Comedy', 'Crime', 'Thriller']",
 "['Action', 'Adventure', 'Comedy', 'Crime']",
 "['Action', 'Adventure', 'Comedy', 'Drama', 'Family', 'War']",
 "['Action', 'Adventure', 'Comedy', 'Drama', 'Fantasy', 'Musical']",
 "['Action', 'Adventure', 'Comedy', 'Drama', 'Music', 'Sci-Fi']",
 "['Action', 'Adventure', 'Comedy', 'Drama', 'Romance', 'Sport']",
 "['Action', 'Adventure', 'Comedy', 'Drama', 'Romance']",
 "['Actio

In [7]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\julia\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\julia\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\julia\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\julia\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\julia\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [8]:
# Inicialización
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

# Función para mapear 
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN 

# Función de preprocesamiento
def preprocess_text_full(text):
    tokens = word_tokenize(text.lower())
    tagged_tokens = pos_tag(tokens)
    
    processed = []
    for word, tag in tagged_tokens:
        if word.isalpha() and word not in stop_words:
            pos = get_wordnet_pos(tag)
            lemma = lemmatizer.lemmatize(word, pos=pos)
            processed.append(lemma)
    return ' '.join(processed)

In [9]:
# Unir title y plot antes del preprocesamiento
dataTraining['text_combined'] = dataTraining['title']  + ' ' + dataTraining['plot']
dataTesting['text_combined'] = dataTesting['title']  + ' ' + dataTesting['plot']

In [10]:
# Aplicar preprocesamiento a la columna combinada
dataTraining['text_clean'] = dataTraining['text_combined'].apply(preprocess_text_full)
dataTesting['text_clean'] = dataTesting['text_combined'].apply(preprocess_text_full)

In [11]:
# Definición de variables predictoras (X)
# Vectorización usando texto combinado limpio
vectorizer = TfidfVectorizer(max_features=20000, ngram_range=(1, 3), min_df=5, max_df=0.8)
X_dtm = vectorizer.fit_transform(dataTraining['text_clean'])

In [12]:
# Definición de variable de interés (y)
dataTraining['genres'] = dataTraining['genres'].map(lambda x: eval(x))
le = MultiLabelBinarizer()
y_genres = le.fit_transform(dataTraining['genres'])

In [13]:
X_train, X_val, y_train, y_val = train_test_split(X_dtm, y_genres, test_size=0.33, random_state=42)

In [14]:
param_grid = {
    'estimator__C': [0.01, 0.1, 1, 10],
    'estimator__penalty': ['l2'],  
    'estimator__solver': ['lbfgs']  
}

clf = OneVsRestClassifier(LogisticRegression(max_iter=1000, random_state=42))
search = RandomizedSearchCV(clf, param_distributions=param_grid, scoring='roc_auc', n_iter=10, cv=3, verbose=1)
search.fit(X_train, y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits


RandomizedSearchCV(cv=3,
                   estimator=OneVsRestClassifier(estimator=LogisticRegression(max_iter=1000,
                                                                              random_state=42)),
                   param_distributions={'estimator__C': [0.01, 0.1, 1, 10],
                                        'estimator__penalty': ['l2'],
                                        'estimator__solver': ['lbfgs']},
                   scoring='roc_auc', verbose=1)

In [15]:
# Predicción del modelo de clasificación
y_pred_val = search.predict_proba(X_val)

# Impresión del desempeño del modelo
roc_auc_score(y_val, y_pred_val, average='macro')

0.8841672294285533

In [16]:
# transformación variables predictoras X del conjunto de test
X_test_dtm = vectorizer.transform(dataTesting['text_clean'])

cols = ['p_Action', 'p_Adventure', 'p_Animation', 'p_Biography', 'p_Comedy', 'p_Crime', 'p_Documentary', 'p_Drama', 'p_Family',
        'p_Fantasy', 'p_Film-Noir', 'p_History', 'p_Horror', 'p_Music', 'p_Musical', 'p_Mystery', 'p_News', 'p_Romance',
        'p_Sci-Fi', 'p_Short', 'p_Sport', 'p_Thriller', 'p_War', 'p_Western']

# Predicción del conjunto de test
y_pred_test_genres = search.predict_proba(X_test_dtm)

In [17]:
# Guardar predicciones en formato exigido en la competencia de kaggle
res = pd.DataFrame(y_pred_test_genres, index=dataTesting.index, columns=cols)
res.to_csv('pred_genres_text_LR6.csv', index_label='ID')
res.head()

,p_Action,p_Adventure,p_Animation,p_Biography,p_Comedy,p_Crime,p_Documentary,p_Drama,p_Family,p_Fantasy,...,p_Musical,p_Mystery,p_News,p_Romance,p_Sci-Fi,p_Short,p_Sport,p_Thriller,p_War,p_Western
1,0.093281,0.102386,0.023432,0.027083,0.419403,0.116994,0.032389,0.572930,0.050329,0.102461,...,0.036655,0.095268,0.000674,0.552203,0.058279,0.012224,0.024273,0.181986,0.025613,0.028351
4,0.131801,0.045823,0.030803,0.134925,0.265452,0.286769,0.063735,0.719451,0.038059,0.032385,...,0.028600,0.043530,0.000700,0.080440,0.030631,0.013528,0.032314,0.217179,0.044493,0.030178
5,0.093493,0.039856,0.016708,0.046594,0.184503,0.534831,0.034807,0.732076,0.025625,0.034907,...,0.022212,0.327201,0.000670,0.175775,0.070992,0.009078,0.022473,0.450213,0.031329,0.022262
6,0.104753,0.079416,0.019257,0.041701,0.239170,0.090025,0.037957,0.682857,0.048160,0.049162,...,0.033952,0.082882,0.000673,0.296932,0.078665,0.009385,0.029040,0.270503,0.069368,0.022386
7,0.066464,0.065708,0.027536,0.028927,0.227548,0.101744,0.038931,0.294596,0.057413,0.118364,...,0.024717,0.148388,0.000683,0.156443,0.342688,0.010848,0.017918,0.307807,0.024253,0.021973
